In [ ]:
import os
import re
import pandas as pd
import numpy as np
import struct
from collections import Counter
import time

In [ ]:
x = [1,2,3,4]
norm = 0
for i in x:
    norm += np.exp(i)
print(np.exp(x)/norm)

In [ ]:
# 载入数据
def load_mnist():
    labels_path = os.path.join('./data/train/train-labels.idx1-ubyte')
    images_path = os.path.join('./data/train/train-images.idx3-ubyte')
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII', imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

In [ ]:
import matplotlib.pyplot as plt
data, label = load_mnist()
img = data[0].reshape(28,28)
plt.imshow(img, cmap='Greys', interpolation='nearest')
plt.title(label[0])
plt.show()
print(data.shape)
# print(X_train[0])
print(data[0].shape)

In [ ]:
train_data = data[:50000,:]
train_label = label[:50000]
val_data = data[50000:,:]
val_label = label[50000:]

In [ ]:
mul_label = np.zeros([len(label),10])
for i in range(len(label)):
    mul_label[i,label[i]] = 1
mul_train_label = mul_label[:50000]
mul_val_label = mul_label[50000:]

In [ ]:
def ReLU(x, deriv=False):
    if deriv == True:
        x[np.where(x > 0)]=1
        x[np.where(x<=0)]=0
        return x
    else:
        mask = x > 0
        return x*mask

In [ ]:
def softmax(z):
    norm = 0
    for i in z:
        norm += np.exp(i)
    return np.exp(z)/norm

In [ ]:
def BatchNorm1d(z):
    s = np.std(z)
    E = z.mean()
    z = (z - E) / ((s + 1e-5) **0.5) 
    return z

In [ ]:
class my_NN(object):
    def __init__(self):
        self.x = np.zeros(784)                         # input输入
        self.w = 0.01 * np.random.randn(100,784)       # I-H权值
        self.netj = np.zeros(100)                      # hidden输入
        self.y = np.zeros(100)                         # hidden输出
        self.v = 0.01 * np.random.randn(10,100)        # H-O权值
        self.netk = np.zeros(10)                       # output输入
        self.z = np.zeros(10)                          # output输出
        self.gradw = np.zeros([100,784])
        self.gradv = np.zeros([10,120])
        self.ita = 1e-3
        self.momentum = 0.9
        
    
    def Forward(self, x):
        self.x = (x/255)*0.99 + 0.01
        self.netj = np.dot(self.w, self.x)
        self.netj = BatchNorm1d(self.netj)
        self.y = ReLU(self.netj)
        
        self.netk = np.dot(self.v, self.y)
        self.netk = BatchNorm1d(self.netk)
        self.z = softmax(ReLU(self.netk))
        
        
    def Backpropagation(self, t):
        delta1 = np.zeros(self.z.shape)        
        delta1 = ((t - self.z) * ReLU(self.netk, True)).T
        self.gradv = self.momentum * self.gradv + self.ita * np.outer(delta1, self.y)
        self.v += self.gradv
        
        delta2 = np.zeros(len(self.y))
        for i in range(len(delta2)):
            delta2[i] = np.dot(delta1, self.v[:, i])
        delta2 = ReLU(delta2 * self.netj, True)
        self.gradw = self.momentum * self.gradw + self.ita * np.outer(delta2, self.x)
        self.w += self.gradw

    def loss(self, label):
        return 0.5 * np.linalg.norm(label - self.z)
    
    def Set(self, w, v):
        self.w = w
        self.v = v

In [ ]:
def fit(epoches, net, train_data, mul_train_label, val_data, val_label):
    history = []
    train_losses = []
    val_acc = []
        
    for epoch in range(epoches):
        loss = 0
        for i in range(len(train_data)):
            # 采用单样本训练法
            net.Forward(train_data[i])            
            net.Backpropagation(mul_train_label[i])

            loss += net.loss(mul_train_label[i])
            # if i%10000 == 0:
            #     print("loss in %d is:%f" %(i,loss))
        # 每轮训练进行一次验证集验证，并将结果记录，便于结束后绘制图像
        acc = evaluate(net, val_data, val_label)
        train_losses.append(loss)
        val_acc.append(acc)
        # 打印验证结果
        print("epoch:%d  train_loss=%f  val_acc=%f" %(epoch, loss, acc))
        net.ita *= 0.99 ** (epoch//2) 

    return train_losses, val_acc

In [ ]:
def evaluate(net, val_data, val_label):
    if len(val_data) == 784:
        net.Forward(val_data)
        if(np.argmax(net.z) == np.argmax(val_label)):
            return True
        else:
            return False
    else:
        acc = 0
        for i in range(len(val_label)):
            net.Forward(val_data[i])
            if np.argmax(net.z) == np.argmax(val_label[i]):
                acc += 1
        return acc/len(val_label)

In [ ]:
net = my_NN()
print(net.w)

In [ ]:
# 5重交叉验证
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

In [ ]:
epoches = 30
train_losses_stack = []
val_acc_stack = []
net_stack = []
ford = 1
for X_train_i, X_test_i in kf.split(data):
    net = my_NN()
    train_losses, val_acc = fit(epoches, net, data[X_train_i], mul_label[X_train_i], \
                                data[X_test_i], mul_label[X_test_i])
    train_losses_stack.append(train_losses)
    val_acc_stack.append(val_acc)
    net_stack.append(net)

In [ ]:
evaluate(net,train_data[100],mul_train_label[100])

In [ ]:
fig, ax = plt.subplots(
    nrows=2,
    ncols=5,
    sharex=True,
    sharey=True, )

ax = ax.flatten()
for i in range(5):
    ax[i].plot(range(0,epoches), train_losses_stack[i])
    ax[i].xlabel("epoch_num")
    ax[i].ylabel("loss")
for i in range(5):
    ax[i+5].plot(range(0,epoches), val_acc_stack[i])
    ax[i+5].xlabel("epoch_num")
    ax[i+5].ylabel("val_acc")
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
for i in range(len(val_acc_stack)):
    print(val_acc_stack[i][-1])

In [ ]:
for i in range(5):
    print(evaluate(net_stack[i],val_data,mul_val_label))

In [ ]:
stack_w = []
stack_v = []
for i in range(5):
    stack_w.append(net_stack[i].w)
    stack_v.append(net_stack[i].v)
net = my_NN()
net.Set(np.array(stack_w).mean(0), np.array(stack_v).mean(0))

In [ ]:
def predict(net, data):
    predict = np.zeros(len(data))
    for i in range(len(data)):
        net.Forward(data[i])
        predict[i] = np.argmax(net.z)
        print("No. %d is %f" %(i,predict[i]))
    return predict

In [ ]:
from numpy import genfromtxt
# 测试集预测

test_data = genfromtxt('data/test_data.csv',delimiter=',')
test_data = test_data[1:,:784]
print(test_data.shape)

In [ ]:
predict = predict(net, test_data)

In [ ]:
# 写入excel
# 记录测试结果
import csv
with open("task_test_label.csv","w", newline="") as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["index","label"])
    for i in range(len(predict)):
        writer.writerow([i,int(predict[i])])

In [ ]:
img = test_data.reshape(10000,28,28)
num = 4
plt.imshow(img[num], cmap='Greys', interpolation='nearest')
plt.title(predict[num])
plt.show()

In [ ]:
num = 100
plt.imshow(img[num], cmap='Greys', interpolation='nearest')
plt.title(predict[num])
plt.show()